In [ ]:
1 Виртуальное окружение python(3) venv -m folder
2 source/bin/activate  ### folder\Scripts\activate.bat
3 Установка зависимостей pip install -r requirements.txt
4 Созание django  django-admin startproject  @name
5 Создание приложения python manage.py startapp @name
6 Запуск Сервера: python manage.py runserver
в django3 появился asgi - наследник wsgi это стандарт как для синхронных так и для асинхронных приложений
Паттерн Model View Template - Model View Controller

Urls.py- представляет собой роутинг запросов/ связывает url и приложения
1 Создание таблицы баззы данных
sudo -u postgres psql postgres - run root user
\password postgres - pass admin

 create user user_name with password 'password';
    alter role user_name set client_encoding to 'utf8';
    alter role user_name set default_transaction_isolation to 'read committed';
    alter role user_name set timezone to 'UTC';
    
Создание бд для джанго проекта:
create database django_db owner user_name;

В окружении проекта устанавливаем бэкэнд для PostgreSQL

pip install psycopg2

Последний наш шаг - настроить раздел DATABASES конфигурационного файла проекта settings.py

'ENGINE': 'django.db.backends.postgresql_psycopg2',
'NAME': 'django_db',
'USER' : 'user_name',
'PASSWORD' : 'password',
'HOST' : '127.0.0.1',
'PORT' : '5432',

In [ ]:
После добавления моделей Зарегистрируем их в admin.py
from django.contrib import admin
from .models import *

admin.site.register(Category)
admin.site.register(Genre)
admin.site.register(Movie)
admin.site.register(Actors)
admin.site.register(MovieShots)
admin.site.register(Rating)
admin.site.register(RatingStar)
admin.site.register(Reviews)


In [ ]:
Создадим суперпользователя:
python manage.py createsuperuser
rustam
1514150Ee
в settings.py language_code = 'ru'


In [ ]:
Для того, чтобы картинки грузились в одну папку отредактируем settings.py
MEDIA_URL = '/media/'
MEDIA_ROOT = os.path.join(BASE_DIR, 'media')
в корневой папке оздается папка media
{% for img in item.imageofitem_set.all %}
    <img src="{{ img.picture.url }}"><br>
{% endfor %}
Чтобы в режиме DEBUGсервак раздавал картинки, нужно в корневой папке в urls.py добавить:
from django.conf import settings
from django.conf.urls.static import static

if settings.DEBUG:
    urlpatterns += static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)


In [ ]:
Редактируем Views
from django.views.generic.base import View
from .models import Movie
Простейшая реальзация состоит из функций/
Но лучше сразу определять классы
from django.shortcuts import render

class MoviesView(View):
	'''List of movie'''
	def get(self, request):
		movies = Movie.objects.all()
		data = {'movie_list': movies}
		return render(request, 'movies/movie_list.html' context=data)

In [ ]:
в папке templates создаем  шаблон 

в других шаблонах расширяем его: {% extends 'base.html' %}
    также в базовом шаблоне прописывае тэги
    {% block title %}{% endblock %}
    {% block content %}{% endblock %}
куда встяавляем содержимое 

перенаправляем url 
создадим url  в самом приложении
from django.urls import path
from . import views
urlpattern = [
    path('', views.MovieView.as_view()) - Главная страница направляется в класс MovieView
]


в файле urls.py корневой папки Django определим маршрутизацию на urls нашего приложения
from django.conf import settings
from django.conf.urls.static import static
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path("", include("movies.urls")),
]



In [ ]:
Поключим шаблон
В корне проекта создадим директорию STATIC
укажим папку в настройки settings
STATIC_DIR=os.path.join(BASE_DIR, 'static')
STATICFILES_DIRS=[STATIC_DIR]

In [ ]:
В шаблоне укажем {% load static %}
в ссылках href укажем {% static путь к директории и самому файлу не указывая на статик папку %}

Переход на страницу одного фильма, поста!
для перехода на страницу фильма нужно прописать slug или pk  ссылку
urlpattern = [
    path('', views.MovieView.as_view()) - Главная страница направляется в класс MovieView
    path('<slug:slug>/' views.MovieDetailView.as_view)
    path('<ind:pk>/')
    
]


Данный slug будет передан в метод get views
на ссылке можно сделать так:
    <a href='/{{ movie.url }}/'
    или так:
    <a href='/{% url 'movie_detail' {{movie.url}} %}/' а в urls указать name='movie_detail'
    Более правильный способ:
        в models.py в классе таблицы указать 
        def get_absolute_url(self):
            return reverse('movie_detail', kwargs= {'slug': self.url})
        а в теге <a href='{{ movie.get_absolute_url }}'
    

In [ ]:
Во views.py записываем 2 класса
class MoviesView(View):
    def get(self, request):
        movies = Movie.objects.all()
        return render(request, 'movies/movies.html', {'movie_list': movies})
    
class MovieDetailView(View):
    def get(self, request, slug):
        movie = Movie.objects.get(url=slug)
        return render(request, 'movies/movie_detail.html', {'movie': movie})
    
Данные классы можно заменить следующими классами

from django.views.generic import ListView, DetailView
from django.views.generic.base import View

class MoviesView(ListView):
    """Список фильмов"""
    model = Movie - модель
    queryset = Movie.objects.filter(draft=False)
    template_name = 'movies/movies.html'

class MovieDetailView(DetailView):
    """Полное описание фильма"""
    model = Movie
    slug_field = "url"
    
    тут мы не указали шаблон -т.к. он автоматически подставляет суффикс detail в модель movie
